# Отчет по выполнению домашнего задания #2

__Дата выдачи__: 24.02.2016

__Дедлайн__: 23:59 9.03.2016

__Выполнил__: Булгаков Дмитрий (ИАД16)

In [39]:
import numpy as np
import sklearn as sk
import os
import string
import re

In [40]:
def convert_text(s):
    # Removes all characters from string except letters and digits and convert letters to lowercase
    return re.sub("[^a-zA-Z0-9]", " ", s.lower())

def read_txts(dir_path="./txt_sentoken/pos/"):
    # Reads all files from directory
    if dir_path[-1] != "/":
        dir_path = dir_path + "/"
    txt_list = []
    for file in os.listdir(dir_path):
        file = dir_path + file
        fin = open(file, 'r')
        txt = " ".join(fin.readlines())
        txt = convert_text(txt)
        txt_list.append(txt)
    return txt_list

In [41]:
class PoissonNB:
    def __init__(self, class_prior=None):
        """
        class_prior : np.array, size (n_classes,)
        Prior probabilities of the classes. If specified the priors are not
        adjusted according to the data.
        """
    
    def fit(self, X, y, epsilon=1e-9):
        """
        Fit Poisson Naive Bayes according to X, y
        
        Parameters
        ----------
        X : np.array, shape (n_samples, n_features)
            Training vectors, where n_samples is the number of samples
            and n_features is the number of features.
        y : np.array, shape (n_samples,)
            Target values.
        """
        
    def predict(self, X):
        """
        Perform classification on an array of test vectors X.
        
        Parameters
        ----------
        X : np.array, shape = [n_samples, n_features]
        
        Returns
        -------
        C : np.array, shape = [n_samples]
            Predicted target values for X
        """

__Решение задачи:__

__#1.__ Загрузите и преобразуйте данные с помощью функции read_txts() из выданного ноутбука. В итоге

должно получиться два списка: с положительными и с отрицательными рецензиями.

In [42]:
pos_rev = read_txts("./txt_sentoken/pos/")
neg_rev = read_txts("./txt_sentoken/neg/")

__#2__ Из всех рецензий сформируйте два списка: тексты для обучающей выборки (по 700 случайных каж-

дого класса) и для контрольной выборки (по 300 оставшихся), а также вектор правильных ответов

для обучающей и контрольной выборки. Например, положительные рецензии можно относить к

классу «1», а отрицательные — к классу «0».

In [91]:
X = np.hstack((pos_rev, neg_rev))
y = np.ones(len(pos_rev) + len(neg_rev))
y[len(pos_rev):] = 0
X_train, X_test, y_train, y_test = sk.cross_validation.train_test_split(X, y, test_size=0.3, random_state=0)

__#3__ Прочитайте, как работает класс sklearn.feature_extraction.text.CountVectorizer, и с его помощью со-

здайте две матрицы «объекты × признаки»: для обучающей и контрольной выборки. Учтите, что

CountVectorizer.transform возвращает разреженную матрицу — чтобы преобразовать её к знакомому

нам np.array, воспользуйтесь функцией .toarray().

In [106]:
cv = sk.feature_extraction.text.CountVectorizer()
train_features = cv.fit_transform(X_train).toarray()
test_features = cv.fit_transform(X_test).toarray()

__#4__ Сами реализуйте класс PoissonNB, реализующий пуассоновский наивный байесовский классифи-

катор. Методы, которые должны быть реализованы в этом классе, описаны в jupyter ноутбуке,

выданном вместе с заданием.

__#5__Протестируйте ваш классификатор на данных и посчитайте accuracy — долю правильных ответов.